In [ ]:
import pandas as pd

data = pd.read_csv('research_abstract_labeled.csv', sep=";", encoding='cp1252')

formula_titles = data[data.text.str.contains('$', regex=False, na=False)].drop_duplicates(subset=['title'], keep='first').title.tolist()

filtered = data[data.title.isin(formula_titles) != True]

In [ ]:
import pandas as pd
from generated_text_detector.utils.model.roberta_classifier import RobertaClassifier
from generated_text_detector.utils.preprocessing import preprocessing_text
from transformers import AutoTokenizer
import torch.nn.functional as F
import torch
from tqdm import tqdm

# Load model and tokenizer
model = RobertaClassifier.from_pretrained("SuperAnnotate/ai-detector")
tokenizer = AutoTokenizer.from_pretrained("SuperAnnotate/ai-detector")
model.eval()

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Assume `data` is your DataFrame with a column named "text"
# Parameters
batch_size = 32 
texts = filtered["text"].tolist()
all_probs = []

# Process in batches
for i in tqdm(range(0, len(texts), batch_size), desc="Processing"):
    batch_texts = texts[i:i + batch_size]
    
    # Preprocess and tokenize all in batch
    preprocessed = [preprocessing_text(t) for t in batch_texts]

    tokens = tokenizer(
        preprocessed,
        add_special_tokens=True,
        max_length=512,
        padding=True,
        truncation=True,
        return_token_type_ids=True,
        return_tensors="pt"
    )

    # Move tokens to device
    tokens = {key: val.to(device) for key, val in tokens.items()}

    with torch.no_grad():
        _, logits = model(**tokens)
        probs = F.sigmoid(logits).squeeze(1).tolist()

    all_probs.extend(probs)

# Add probabilities to DataFrame
filtered["ai_probability"] = all_probs


c:\Users\User\Documents\egyetem\szakdolgozat\venv311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing: 100%|██████████| 400/400 [9:09:49<00:00, 82.47s/it]  
C:\Users\User\AppData\Local\Temp\ipykernel_12408\3336366991.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["ai_probability"] = all_probs


In [ ]:
#filtered.to_csv('szakdolgozat_detector.csv')